In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Softmax, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import random
from tensorflow.keras import backend as K
import torch


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import os

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
tf.__version__

'2.12.0'

# **Data process**

In [4]:

df = open('/kaggle/input/go-competition/29_Training Dataset/Training Dataset/dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

random.seed(42)
random.shuffle(games)



In [5]:
games_sorted = sorted(games, key=lambda x: len(x), reverse=True)

In [6]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

In [7]:
def prepare_input(moves):
    x = np.zeros((19,19,3))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            #x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            #x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,2] = 1
    #x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [8]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 100160, Total Moves: 22853380


In [9]:
import random

random_sample = random.sample(games, 500)


x = []
y = []
x1 = [] 
y1 = []
for game in random_sample:
        x1 = prepare_input_Zero1(game)
        x.extend(x1)
        y1 = prepare_label_Zero1(game)
        y.extend(y1)
x = np.array(x)
y = np.array(y)
y_one_hot = tf.one_hot(y, depth=19*19)

In [10]:
x = []
y = []
for game in games_sorted[:500]:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

y_one_hot = tf.one_hot(y, depth=19*19)


In [11]:
x.shape

(171704, 19, 19, 3)

In [12]:
y.shape

(171704,)

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.05)

In [14]:
def residual_block(x, num_filters):
    # 保存輸入，用於殘差連接
    shortcut = x
    shortcut = Conv2D(num_filters, kernel_size=3, padding='same')(shortcut)
    # 第一個卷積層
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    # 第二個卷積層
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    # 殘差連接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [15]:
def residual_1(x, num_filters):
    # 保存輸入，用於殘差連接
    shortcut = x
    
    # 第一個卷積層
    x = Conv2D(num_filters, kernel_size=1, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters/2, kernel_size=1, padding='same')(x)
    x = Activation('relu')(x)
    # 第二個卷積層
    x = Conv2D(num_filters, kernel_size=1, padding='same')(x)
    # 殘差連接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [16]:
def create_model():
    inputs = keras.Input(shape=(19, 19, 3))
    x = inputs

    outputs = Conv2D(kernel_size=3, filters=128, padding='same', activation='relu')(x)

    for _ in range(6):  # 您可以根據需要調整殘差塊的數量
        outputs = residual_block(outputs, num_filters=128)
    outputs = residual_1(outputs,128)
    
    for _ in range(6):  # 您可以根據需要調整殘差塊的數量
        outputs = residual_block(outputs, num_filters=128)
    outputs = residual_1(outputs,128)
    
    outputs = Conv2D(kernel_size=1, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [17]:
from tensorflow.keras import mixed_precision
strategy = tf.distribute.MirroredStrategy()
mixed_precision.set_global_policy('mixed_float16')

from tensorflow.keras.optimizers.schedules import ExponentialDecay

initial_learning_rate = 0.000006
min_learning_rate = 0.000001
decay_steps = 5000
decay_rate = 0.9

lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)


with strategy.scope():
    #model = create_model()
    # Load the pre-trained model
    model = load_model('/kaggle/input/model-for-train-in-public/model_dan_testv2_6times_0.5288.h5', compile=False)

    # Define the new optimizer
    new_optimizer =  Adam(learning_rate=lr_schedule)
    new_optimizer = mixed_precision.LossScaleOptimizer(new_optimizer)
    # Compile the model with the new optimizer
    model.compile(optimizer=new_optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.trainable = False
# Display the model summary after recompiling
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19, 19, 3)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 19, 19, 128)  3584        ['input_1[0][0]']                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 19, 19, 128)  147584      ['conv2d_1[0][0]']               
                                                                                                  
 activation (Activation)        (None, 19, 19, 128)  0           ['conv2d_3[0][0]']               
                                                                                              

In [18]:

#from tensorflow.keras.callbacks import EarlyStopping
#
#early_stopping = EarlyStopping(
#    monitor='val_loss', 
#    patience=5,           
#    restore_best_weights=True,  
#)

#history = model.fit(
#    x = x_train, 
#    y = y_train,
#    batch_size = 128,
#    epochs = 50,
#    validation_data=(x_val, y_val),
#    callbacks=[early_stopping],
#)


In [19]:
from tensorflow.keras.utils import Sequence
import multiprocessing

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
    # Define your custom data generator class
class MyDataGenerator(Sequence):
    def __init__(self, games, batch_size):
        self.games = games
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.games) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_games = self.games[start:end]

        x, y_one_hot = data_pocess(batch_games)
        indices = np.random.permutation(len(x))
        x = x[indices]
        y_one_hot = y_one_hot[indices]
        return x, y_one_hot

In [20]:
def data_pocess(batch_games):
    with strategy.scope():
        x = []
        y = []
        for game in batch_games:
            moves_list = game.split(',')
            game_x = []
            game_y = []
            for count, move in enumerate(moves_list):
                game_x.append(prepare_input(moves_list[:count]))
                game_y.append(prepare_label(moves_list[count]))
            x.extend(game_x)
            y.extend(game_y)
        x = np.array(x)
        y = np.array(y)
        y_one_hot = tf.one_hot(y, depth=19 * 19)
        return x , y_one_hot.numpy()

In [21]:

x_train =  games_sorted[40000:70000] 
x_val =  random_sample 

x_train = np.array(x_train)
x_val = np.array(x_val)

random.shuffle(x_train)
random.shuffle(x_val)


In [22]:
print("model_dan_testv2_7times")

model_dan_testv2_7times


In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint(
    filepath='model_dan_testv2_7times.h5',  
    save_best_only=True,  
    verbose=1  
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Create an instance of the data generator
batch_size = 8
data_generator_train = MyDataGenerator(x_train, batch_size)
data_generator_val = MyDataGenerator(x_val, batch_size)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=2,          
    restore_best_weights=True, 
)

# Train your model using the data generator
history = model.fit(
    data_generator_train,
    epochs=8,
    validation_data=data_generator_val,
    callbacks=[model_checkpoint, early_stopping],
    workers = 2
)

Epoch 1/8
  13/3750 [..............................] - ETA: 5:51:33 - loss: 1.7112 - accuracy: 0.5272

In [ ]:
model.save('./model_dan_testv2_7times.h5')